In [ ]:
!pip install cartopy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.7/11.7 MB 59.7 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import  xarray as xr
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime,timedelta
import warnings
import math
from tqdm.auto import tqdm

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import cartopy.feature as cf
import cartopy.crs as ccrs
import cartopy.mpl.ticker as cticker

from cartopy.util import add_cyclic_point
from matplotlib import animation
from tqdm.auto import tqdm
from datetime import datetime,timedelta

from matplotlib.cm import get_cmap
warnings.filterwarnings("ignore")

In [ ]:
#@title File check
import os
from tqdm.auto import tqdm
directory_path='/content/drive/MyDrive/AP_HW/ERA5 Data/temperature_levels(1000-900)'
length=len(sorted(os.listdir(directory_path)))
print('number of files:',length)
for filename in tqdm(sorted(os.listdir(directory_path))):
  if filename.endswith('.nc'):
    file_path=os.path.join(directory_path,filename)
    ds=xr.open_dataset(file_path)
    year=str(ds.valid_time.dt.year.values[1])
    length=len(sorted(os.listdir(directory_path)))
    file_year=filename[-7:-3]
    if file_year!=year: print(filename)

number of files: 54


  0%|          | 0/54 [00:00<?, ?it/s]

In [ ]:
ds=xr.open_mfdataset('/content/drive/MyDrive/AP_HW/ERA5 Data/temperature_levels(1000-900)/temp(1000-900)-1971.nc')
ds

<xarray.Dataset> Size: 1GB
Dimensions:         (valid_time: 2920, pressure_level: 5, latitude: 133,
                     longitude: 153)
Coordinates:
    number          int64 8B ...
  * valid_time      (valid_time) datetime64[ns] 23kB 1971-01-01 ... 1971-12-3...
  * pressure_level  (pressure_level) float64 40B 1e+03 975.0 950.0 925.0 900.0
  * latitude        (latitude) float64 1kB 38.0 37.75 37.5 ... 5.5 5.25 5.0
  * longitude       (longitude) float64 1kB 60.0 60.25 60.5 ... 97.5 97.75 98.0
    expver          (valid_time) <U4 47kB dask.array<chunksize=(2920,), meta=np.ndarray>
Data variables:
    t               (valid_time, pressure_level, latitude, longitude) float32 1GB dask.array<chunksize=(974, 1, 45, 51), meta=np.ndarray>
Attributes:
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2025-01-05T19:48 GRIB to CDM+CF via cfgrib-0.9.1...

In [ ]:
#@title Climatology Calculation - MAM
import os
from tqdm.auto import tqdm
import datetime

# Print process start time
print(datetime.datetime.utcnow() + datetime.timedelta(hours=6))

directory_path = '/content/drive/MyDrive/AP_HW/ERA5 Data/temperature_levels(1000-900)'

# List to store yearly datasets
mean_list = []

# Loop over files
for filename in tqdm(sorted(os.listdir(directory_path))):
    if filename.endswith('.nc'):
        # Open dataset
        file_path = os.path.join(directory_path, filename)
        year = filename[-7:-3]
        ds = xr.open_dataset(file_path)

        # Define MAM period
        start_time = str(np.datetime64(f'{year}-03-01T00:00:00.000000000') - np.timedelta64(6, 'h'))
        end_time = str(np.datetime64(f'{year}-05-31T17:00:00.000000000'))
        time_range = slice(start_time, end_time)
        ds = ds.sel(valid_time=time_range)

        # Adjust times to Bangladesh timezone
        ds['valid_time'] = ds['valid_time'] + np.timedelta64(6, 'h')

        # Yearly mean
        ds = ds.groupby('valid_time.year').mean(dim='valid_time', skipna=True)

        # Add to list
        mean_list.append(ds)

# Concatenate all years
temp_ds = xr.concat(mean_list, dim='year')

# Compute climatology
temp_clim_ds = temp_ds.sel(year=slice(1981, 2010))
temp_climatology = temp_clim_ds.mean(dim='year', skipna=True)

# Save to NetCDF
temp_climatology.to_netcdf('/content/drive/MyDrive/AP_HW/Scripts-ll/climatology/data/shum&temp/temp_climatology(1000-900)-mam.nc')

# Print process end time
print(datetime.datetime.utcnow() + datetime.timedelta(hours=6))


2025-06-21 11:26:44.252529


  0%|          | 0/54 [00:00<?, ?it/s]

2025-06-21 11:37:15.485031


In [ ]:
#@title Climatology Calculation - JJAS
import os
from tqdm.auto import tqdm
import datetime

# Print process start time
print(datetime.datetime.utcnow() + datetime.timedelta(hours=6))

directory_path = '/content/drive/MyDrive/AP_HW/ERA5 Data/temperature_levels(1000-900)'

# List to store yearly datasets
mean_list = []

# Loop over files
for filename in tqdm(sorted(os.listdir(directory_path))):
    if filename.endswith('.nc'):
        # Open dataset
        file_path = os.path.join(directory_path, filename)
        year = filename[-7:-3]
        ds = xr.open_dataset(file_path)

        # Define JJAS period
        start_time = str(np.datetime64(f'{year}-06-01T00:00:00.000000000') - np.timedelta64(6, 'h'))
        end_time = str(np.datetime64(f'{year}-09-30T17:00:00.000000000'))
        time_range = slice(start_time, end_time)
        ds = ds.sel(valid_time=time_range)

        # Adjust times to Bangladesh timezone
        ds['valid_time'] = ds['valid_time'] + np.timedelta64(6, 'h')

        # Yearly mean
        ds = ds.groupby('valid_time.year').mean(dim='valid_time', skipna=True)

        # Add to list
        mean_list.append(ds)

# Concatenate all years
temp_ds = xr.concat(mean_list, dim='year')

# Compute climatology
temp_clim_ds = temp_ds.sel(year=slice(1981, 2010))
temp_climatology = temp_clim_ds.mean(dim='year', skipna=True)

# Save to NetCDF
temp_climatology.to_netcdf('/content/drive/MyDrive/AP_HW/Scripts-ll/climatology/data/shum&temp/temp_climatology(1000-900)-jjas.nc')

# Print process end time
print(datetime.datetime.utcnow() + datetime.timedelta(hours=6))


2025-06-21 11:37:15.500765


  0%|          | 0/54 [00:00<?, ?it/s]

2025-06-21 11:46:12.455518
